In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas
import itertools
import numpy as np
from tqdm import tqdm_notebook
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from config import IAMOnDo_config

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
config = IAMOnDo_config()

In [4]:
from preprocess import read_data

train_features, train_labels = read_data(os.path.join(config.data_dir, "train"))
valid_features, valid_labels = read_data(os.path.join(config.data_dir, "valid"))
test_features, test_labels = read_data(os.path.join(config.data_dir, "test"))

In [5]:
train_batch_list = zip(train_features, train_labels)
valid_batch_list = zip(valid_features, valid_labels)
test_batch_list = zip(test_features, test_labels)
data = (train_batch_list, valid_batch_list, test_batch_list)

In [6]:
from strokeFeature import logistic
from utils import batchify

model = logistic(config)
cur_batch = batchify(train_batch_list[0:5], gpu=config.gpu)
out = model(cur_batch)
print(out.size())

torch.Size([5, 547, 6])


In [7]:
from torch import optim
from train import train, neg_log_likelihood_loss
from strokeFeature import logistic

config.batch_size = 10
config.num_epoch = 10
config.lr = 1e-3
config.gpu = True

model = logistic(config)

optimizer = optim.Adam(model.parameters(),
                      lr=config.lr)

train(data, model, optimizer, neg_log_likelihood_loss, config)

/home/junyu/anaconda2/lib/python2.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
train.py:65: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(model.parameters(), config.clip_value)


Iteration  1 | Loss: 506.885986 | var_norm: 6.733 | learning rate: 0.001000


Iteration  2 | Loss: 440.811859 | var_norm: 6.927 | learning rate: 0.000952


Iteration  3 | Loss: 434.748291 | var_norm: 7.153 | learning rate: 0.000909


Iteration  4 | Loss: 426.740295 | var_norm: 7.360 | learning rate: 0.000870


KeyboardInterrupt: 